# Выбор локации для скважины

## Загрузка и подготовка данных

In [67]:
#выгружаем библиотеки
import pandas as pd
import numpy as np
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [68]:
# читаем файлы
data1 = pd.read_csv('/datasets/geo_data_0.csv')
data2 = pd.read_csv('/datasets/geo_data_1.csv')
data3 = pd.read_csv('/datasets/geo_data_2.csv')

In [69]:
data1.info(), data2.info(), data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8

(None, None, None)

In [70]:
display(data1.head())
display(data2.head())
data3.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Колонка id - не является нужным признаком для построение модели, однако там содержиться название скважины, поэтому сохраним id в виде индексов датасета.

In [71]:
data1.set_index('id', inplace=True)
data2.set_index('id', inplace=True)
data3.set_index('id', inplace=True)

In [72]:
# обучения и предсказания модели
def get_target(data):
    features = data.drop(['product'], axis =1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=100)
    return target_valid

def training(data):
    features = data.drop(['product'], axis =1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=100)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    return predictions
    
def metrics(data, predictions): 
    features = data.drop(['product'], axis =1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=100)
    mean_predicted_product = predictions.mean()
    rmse = (mean_squared_error(target_valid,predictions))**0.5
    print(f'Средний запас предсказанного сырья: {mean_predicted_product:.2f} тыс. барралей')
    print(f"Метрика RMSE: {rmse:.2f}")  
    

In [73]:
# смотрим первый регион
target1 = get_target(data1)
predictions1=training(data1)
metrics(data1, predictions1)


Средний запас предсказанного сырья: 92.36 тыс. барралей
Метрика RMSE: 37.70


Средний обьем запасов региона около 92 тыс. барралей. Метрика MSE очень большая и достигает 38 на обьект

In [74]:
target2 = get_target(data2)
predictions2=training(data2)
metrics(data2, predictions2)

Средний запас предсказанного сырья: 68.77 тыс. барралей
Метрика RMSE: 0.89


Средний обьем региона 2 уже скромнее, но в тоже время ошибка практически отсуствует

In [75]:
target3 = get_target(data3)
predictions3=training(data3)
metrics(data3, predictions3)
    

Средний запас предсказанного сырья: 94.91 тыс. барралей
Метрика RMSE: 40.11


Тут такая же ситуация как и с первым регионом сквожины всреднем по мнению модели имеют большее количество нефти, однако MSE уже составляет около 40 тыс. барралей.

Регионы 1 и 3 богаче региона 2 в среднем, однако метрики MSE у них гораздо хуже, а значит у них и больший риск убытка.

## Подготовка к расчёту прибыли

In [76]:
# выпишим ключевые переменные для расчета прибыли
budget = 10000000000
well_scouting = 500
well_developing = 200
price_for_barell = 450000
probability_losses = 0.025

In [77]:
# расчитаем минимальное средние количество запасов нефти
# разделим бюджет на цену тыс. барралей и на количество скважин для разработки
min_mean_value = budget/price_for_barell/well_developing
min_mean_value

111.11111111111111

Выходит чтобы покрыть расходы на бюджет нужно, чтобы каждая из 200 скважин в среднем имела запас 111,12 тыс. барралей. Учитывая данные полученные выше по регионам, необходимо более детально отобрать наиболее богатые скважины, иначе данный проект может стать убыточным.

In [78]:
#Пропишем функцию для расчета прибыли
def revenue(selected, price_for_barell, budget):
    revenue = selected[:200].sum()*price_for_barell-budget
    return revenue


## Расчёт прибыли и рисков 

In [91]:
# расчитаем распределение прибыли по регионам с помощью Bootstrap:
def count_revenue(predictions, target):
    state = np.random.RandomState(100)
    values = []
    for i in range(1000):
        predictions = pd.Series(predictions, index=target.index)
        wells_for_scouting = predictions.sample(n=well_scouting, random_state=state, replace=True)
        wells = wells_for_scouting.sort_values(ascending=False)  
        selected = target[wells.index]
        x = revenue(selected, price_for_barell, budget)
        values.append(x.mean()/1000000)
    return values
    
 

In [92]:
values1 = count_revenue(predictions1, target1)
values1=pd.Series(values1)
values1.mean()

473.6865498309988

Средняя прибыль региона 1 около 474 млн. р. 

In [110]:
#найдем доверительный интервал месторождений 1 региона
profit1 = target1*price_for_barell/1000000
print(profit1.quantile(0.025))
profit1.quantile(0.975)

6.21590654734547


77.39498916026398

In [93]:
#оценим убыточность для этого найдем 0.025 квантиль
values1.quantile(probability_losses)

-38.70556500310792

Квантиль отрицательный - регион имеет вероятность убытков свыше 2,5%

In [94]:
# расчитаем для 2 региона
values2 = count_revenue(predictions2, target2)
values2=pd.Series(values2)
values2.mean()

441.0958734877939

Средняя прибыль региона 2 немного ниже 440 млн р.

In [111]:
profit2 = target2*price_for_barell/1000000
print(profit2.quantile(0.025))
profit2.quantile(0.975)

0.0


62.07543348340754

In [95]:
#оценим убыточность для этого найдем 0.025 квантиль
values2.quantile(probability_losses)

52.31096479856335

Квантиль положительный регион не убыточный.

In [96]:
# расчитаем для 3 региона
values3 = count_revenue(predictions3, target3)
values3=pd.Series(values3)
values3.mean()

377.52432889425586

In [112]:
profit3 = target3*price_for_barell/1000000
print(profit3.quantile(0.025))
profit3.quantile(0.975)

5.690919525508897


80.45185089788957

Регион 3 имеет намного ниже параметры. Средняя прибыль 378 млн.р. 

In [97]:
#оценим убыточность для этого найдем 0.025 квантиль
values3.quantile(probability_losses)

-173.8663137285789

Квантиль отрицательный регион убыточный

При анализе только регион 2 имеет низкую степень убыточности. При анализе Boostrap с использованием самых дорогих точек этот регион обладает высоким результатом для нефтедобывающего проекта. Также стоит отметить, что метрика MSE  у региона 2 была ниже всех, значит прогноз региона модель делает с хорошей надежностью.